### Download all the necessary libraries

In [43]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import math
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Libraries to read HTML files
from lxml import html

# Version not working
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

! pip install geocoder
import geocoder # import geocoder


#import folium # map rendering library

print('Libraries imported.')


Libraries imported.


### 1. Download and Scrape Toronto neighbourhoods data set from the Web

Using the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [46]:
# Get the boroughs from ZIP's table on wiki site
toronto_pages = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tzips = pd.DataFrame(toronto_pages[0])
print('Original Data Set: ')
print(tzips.shape)
print(tzips.head(30))
print(tzips.tail())

Original Data Set: 
(287, 3)
   Postcode           Borough     Neighbourhood
0       M1A      Not assigned      Not assigned
1       M2A      Not assigned      Not assigned
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M6A        North York  Lawrence Heights
6       M6A        North York    Lawrence Manor
7       M7A  Downtown Toronto      Queen's Park
8       M8A      Not assigned      Not assigned
9       M9A         Etobicoke  Islington Avenue
10      M1B       Scarborough             Rouge
11      M1B       Scarborough           Malvern
12      M2B      Not assigned      Not assigned
13      M3B        North York   Don Mills North
14      M4B         East York  Woodbine Gardens
15      M4B         East York     Parkview Hill
16      M5B  Downtown Toronto           Ryerson
17      M5B  Downtown Toronto   Garden District
18      M6B        North York         Glencairn
19      M7B

### 1.1 Data wrangling

In [47]:
# 2.1 Ignoring Cells with Borough Not Assigned
tzips_wrg = tzips[tzips['Borough'] != 'Not assigned']
tzips_wrg = tzips_wrg.sort_values(['Postcode'], ascending = (True))
tzips_wrg = tzips_wrg.reset_index(drop=True)
print('Data Set with Borough cleaned')
print('Shape: ')
print(tzips_wrg.shape)
print('Head: ')
print(tzips_wrg.head(10))
print('Tail: ')
print(tzips_wrg.tail(10))

# 2.2 Checking neighbourhood not assigned

for post_x in range(0,len(tzips_wrg)):

    keynbh = tzips_wrg.loc[post_x, 'Neighbourhood']
    
    if keynbh  == '':
        print (post_x, ' ,No Neigh..')
        tzips_wrg.loc[post_x,'Neighbourhood']=tzips_wrg.loc[post_x,'Borough']
          
# 2.3 Merging equal Post code
# define the new dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe (empty only with column names)
tzips_wrg1 = pd.DataFrame(columns=column_names)
tzips_wrg = tzips_wrg.sort_values(['Postcode'], ascending = (True))
Mrgn=0
post_xpr = 0
for post_x in range(0,len(tzips_wrg)):

    keywrg = tzips_wrg.loc[post_x, 'Postcode']
    
    if keywrg  != post_xpr:
        tzips_wrg1 = tzips_wrg1.append(tzips_wrg.loc[post_x])
        mrgn=0
    else:
        mrgn= mrgn + 1
        tzips_wrg1.loc[post_x-mrgn,'Neighbourhood']=tzips_wrg1.loc[post_x-mrgn,'Neighbourhood']+', '+tzips_wrg.loc[post_x, 'Neighbourhood']
        
    post_xpr = keywrg   

Data Set with Borough cleaned
Shape: 
(210, 3)
Head: 
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough      Port Union
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough  Highland Creek
5      M1E  Scarborough       Guildwood
6      M1E  Scarborough     Morningside
7      M1E  Scarborough       West Hill
8      M1G  Scarborough          Woburn
9      M1H  Scarborough       Cedarbrae
Tail: 
    Postcode    Borough         Neighbourhood
200      M9R  Etobicoke  Martin Grove Gardens
201      M9V  Etobicoke        Albion Gardens
202      M9V  Etobicoke      Beaumond Heights
203      M9V  Etobicoke            Humbergate
204      M9V  Etobicoke             Jamestown
205      M9V  Etobicoke           Mount Olive
206      M9V  Etobicoke         South Steeles
207      M9V  Etobicoke           Thistletown
208      M9V  Etobicoke           Silverstone
209      M9W  Etobicoke             Nor

### 1.2. Wrangled Data set number of rows, head and tail information (tzips_wrg1)

In [49]:
tzips_wrg1 = tzips_wrg1.sort_values(['Postcode'], ascending = (True))
tzips_wrg1 = tzips_wrg1.reset_index(drop=True)
print('')
print('Wrangled Data Set')
print('Shape: ')
print(tzips_wrg1.shape)
print('Head: ')
print(tzips_wrg1.head(30))
print('Tail: ')
print(tzips_wrg1.tail(10))




Wrangled Data Set
Shape: 
(103, 3)
Head: 
   Postcode      Borough                                      Neighbourhood
0       M1B  Scarborough                                     Rouge, Malvern
1       M1C  Scarborough             Port Union, Rouge Hill, Highland Creek
2       M1E  Scarborough                  Guildwood, Morningside, West Hill
3       M1G  Scarborough                                             Woburn
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7       M1L  Scarborough                    Golden Mile, Oakridge, Clairlea
8       M1M  Scarborough    Cliffcrest, Scarborough Village West, Cliffside
9       M1N  Scarborough                        Cliffside West, Birch Cliff
10      M1P  Scarborough  Wexford Heights, Dorset Park, Scarborough Town...
11      M1R  Scarborough                     

###  2. Get Latitude and Longitude of the Boroughs postal codes

####  2.1 Trying geolocator.geocode. However it finishes with "service not available" after some iterations. I have decided to use the CSV file.

In [50]:
tzips_wrg1 = tzips_wrg1.reset_index(drop=True)

for nx in range(0,len(tzips_wrg1)):

    post_c = tzips_wrg1.loc[nx, 'Postcode']
          
# initialize your variable to None
    lat_lng_coords = None
    iterx = 0

# loop until you get the coordinates
    while(lat_lng_coords is None):
        address = '{}, Toronto, Ontario'.format(post_c)
        g = geolocator.geocode(address)
        iterx = iterx + 1        
        if iterx >= 3:
           lat_lng_coords =[0,0]
       
    
    tzips_wrg1.loc[nx,'latitude']= lat_lng_coords[0]
    tzips_wrg1.loc[nx,'longitude']= lat_lng_coords[1]
    print(nx)
    
tzips_wrg1.shape

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


GeocoderUnavailable: Service not available

####  2.2 Since "geolocator.geocode" is not responding properly, I will use the CSV file instead

In [51]:
to_coordx = pd.read_csv('http://cocl.us/Geospatial_data')
to_coord = pd.DataFrame(to_coordx).set_index('Postal Code')
to_coord.head(10)


Latitude  Longitude
Postal Code                      
M1B          43.806686 -79.194353
M1C          43.784535 -79.160497
M1E          43.763573 -79.188711
M1G          43.770992 -79.216917
M1H          43.773136 -79.239476
M1J          43.744734 -79.239476
M1K          43.727929 -79.262029
M1L          43.711112 -79.284577
M1M          43.716316 -79.239476
M1N          43.692657 -79.264848

In [52]:
tzips_wrg2=tzips_wrg1
tzips_wrg2['Latitude']=np.nan
tzips_wrg2['Longitude']=np.nan
tzips_wrg2.head()
for nx in range(0,len(tzips_wrg2)):
    pcode = tzips_wrg2.loc[nx,'Postcode']
    tzips_wrg2.loc[nx, 'Latitude']=to_coord.loc[pcode,'Latitude']
    tzips_wrg2.loc[nx, 'Longitude']=to_coord.loc[pcode,'Longitude']

    



In [8]:
print(tzips_wrg2.shape)
tzips_wrg2.head(12)


(103, 5)


Postcode      Borough                                      Neighbourhood  \
0       M1B  Scarborough                                     Rouge, Malvern   
1       M1C  Scarborough             Port Union, Rouge Hill, Highland Creek   
2       M1E  Scarborough                  Guildwood, Morningside, West Hill   
3       M1G  Scarborough                                             Woburn   
4       M1H  Scarborough                                          Cedarbrae   
5       M1J  Scarborough                                Scarborough Village   
6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park   
7       M1L  Scarborough                    Golden Mile, Oakridge, Clairlea   
8       M1M  Scarborough    Cliffcrest, Scarborough Village West, Cliffside   
9       M1N  Scarborough                        Cliffside West, Birch Cliff   
10      M1P  Scarborough  Wexford Heights, Dorset Park, Scarborough Town...   
11      M1R  Scarborough                                  Maryvale, Wexford   

     Latitude  Longitude  
0   43.806686 -79.194353  
1   43.784535 -79.160497  
2   43.763573 -79.188711  
3   43.770992 -79.216917  
4   43.773136 -79.239476  
5   43.744734 -79.239476  
6   43.727929 -79.262029  
7   43.711112 -79.284577  
8   43.716316 -79.239476  
9   43.692657 -79.264848  
10  43.757410 -79.273304  
11  43.750072 -79.295849

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [54]:

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latto = location.latitude
lonto = location.longitude
print('The geograpical coordinates of Toronto City are {}, {}.'.format(latto, lonto))

The geograpical coordinates of Toronto City are 43.653963, -79.387207.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latto, lonto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(tzips_wrg2['Latitude'], tzips_wrg2['Longitude'], tzips_wrg2['Borough'], tzips_wrg2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

NameError: name 'folium' is not defined

In [11]:
#### Define Foursquare Credentials and Version
CLIENT_ID = 'TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14' # your Foursquare ID
CLIENT_SECRET = 'G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14
CLIENT_SECRET:G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY


In [12]:
#### Let's explore the first neighborhood in our dataframe.
print (tzips_wrg2.loc[0, 'Neighbourhood'])

neigh_lat = tzips_wrg2.loc[0, 'Latitude'] # neighborhood latitude value
neigh_lon = tzips_wrg2.loc[0, 'Longitude'] # neighborhood longitude value

neigh_name = tzips_wrg2.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_lon))

Rouge, Malvern
Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lon, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TMIKXOISICLWJJQODBQEWYOC44TQ50DL223HISL2NP0BGO14&client_secret=G1IDMYTRAKU1OP4RW1S35YJJ4OEDYX2UIR44UI3QEJZHNLQY&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [14]:
# Send the GET request and examine the resutls
import json # library to handle JSON files
results = requests.get(url).json()
# results  # too much data
## How is the newyork_data structured?
print ("results structure Dictionary Keys:  ", results.keys())

results structure Dictionary Keys:   dict_keys(['meta', 'response'])


In [15]:
print("response structure Dictionary Keys:  ", results['response'].keys())

response structure Dictionary Keys:   dict_keys(['warning', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])


In [16]:
print("groups structure Dictionary Keys:  ", results['response']['groups'])

groups structure Dictionary Keys:   [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': "Wendy's", 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'}]}]


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
# This line added to understand venues structure
#import json # library to handle JSON files

venues = results['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues) # flatten JSON copy to print below
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Now we are ready to clean the json and structure it into a pandas dataframe.
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
print(nearby_venues.head())

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

(1, 4)
      name            categories        lat        lng
0  Wendy's  Fast Food Restaurant  43.807448 -79.199056
1 venues were returned by Foursquare.


In [19]:
# This line added to understand venues structure
print (venues)


[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bb6b9446edc76b0d771311c', 'name': "Wendy's", 'location': {'crossStreet': 'Morningside & Sheppard', 'lat': 43.80744841934756, 'lng': -79.19905558052072, 'labeledLatLngs': [{'label': 'display', 'lat': 43.80744841934756, 'lng': -79.19905558052072}], 'distance': 387, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4bb6b9446edc76b0d771311c-0'}]


In [20]:
# This line added to understand nearby_venues structure
print(nearby_venues1)

   reasons.count                                      reasons.items  \
0              0  [{'summary': 'This spot is popular', 'type': '...   

                       referralId  \
0  e-0-4bb6b9446edc76b0d771311c-0   

                                    venue.categories  \
0  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...   

                   venue.id venue.location.cc venue.location.city  \
0  4bb6b9446edc76b0d771311c                CA             Toronto   

  venue.location.country venue.location.crossStreet  venue.location.distance  \
0                 Canada     Morningside & Sheppard                      387   

  venue.location.formattedAddress  \
0            [Toronto ON, Canada]   

                       venue.location.labeledLatLngs  venue.location.lat  \
0  [{'label': 'display', 'lat': 43.80744841934756...           43.807448   

   venue.location.lng venue.location.state venue.name  venue.photos.count  \
0          -79.199056                   ON    Wendy's           

## 3. Explore Neighbourhoods in Toronto

In [26]:
# Function to repeat the same process to all the neighbourhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# Code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.
to_venues = getNearbyVenues(names=tzips_wrg2['Neighbourhood'],
                                   latitudes=tzips_wrg2['Latitude'],
                                   longitudes=tzips_wrg2['Longitude']
                                  )

# Let's check the size of the resulting dataframe
print(to_venues.shape)
print(to_venues.head())


# Let's check how many venues were returned for each neighborhood
print(tzips_wrg2.groupby('Neighbourhood').count())

(2230, 7)
                             Neighborhood  Neighborhood Latitude  \
0                          Rouge, Malvern              43.806686   
1  Port Union, Rouge Hill, Highland Creek              43.784535   
2  Port Union, Rouge Hill, Highland Creek              43.784535   
3  Port Union, Rouge Hill, Highland Creek              43.784535   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                           Venue  Venue Latitude  \
0              -79.194353                         Wendy's       43.807448   
1              -79.160497           Royal Canadian Legion       43.782533   
2              -79.160497       Affordable Toronto Movers       43.787919   
3              -79.160497  Scarborough Historical Society       43.788755   
4              -79.188711               G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085              

In [28]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 267 uniques categories.


## 3. Analyze Each Neighborhood

In [31]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

Neighbourhood  Accessories Store  \
0                          Rouge, Malvern                  0   
1  Port Union, Rouge Hill, Highland Creek                  0   
2  Port Union, Rouge Hill, Highland Creek                  0   
3  Port Union, Rouge Hill, Highland Creek                  0   
4       Guildwood, Morningside, West Hill                  0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                   0              0          0           0           0   
1                   0              0          0           0           0   
2                   0              0          0           0           0   
3                   0              0          0           0           0   
4                   0              0          0           0           0   

   Bakery  Bank  Bar  Baseball Field  Baseball Stadium  Basketball Stadium  \
0       0     0    0               0                 0                   0   
1       0     0    1               0                 0                   0   
2       0     0    0               0                 0                   0   
3       0     0    0               0                 0                   0   
4       0     0    0               0                 0                   0   

   Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0      0                0         0           0                   0   
1      0                0         0           0                   0   
2      0                0         0           0                   0   
3      0                0         0           0                   0   
4      0                0         0           0                   0   

   Bike Shop  Bistro  Board Shop  Boat or Ferry  Bookstore  Boutique  \
0          0       0           0              0          0         0   
1          0       0           0              0          0         0   
2          0       0           0              0          0         0   
3          0       0           0              0          0         0   
4          0       0           0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0                0         0             0      

In [32]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()

print(to_grouped.shape)
print(to_grouped)


(100, 268)
                                        Neighbourhood  Accessories Store  \
0                            Adelaide, Richmond, King           0.000000   
1                                           Agincourt           0.000000   
2   Albion Gardens, Beaumond Heights, Humbergate, ...           0.000000   
3     Bathurst Manor, Wilson Heights, Downsview North           0.000000   
4                                     Bayview Village           0.000000   
5                   Bedford Park, Lawrence Manor East           0.000000   
6                                         Berczy Park           0.000000   
7   Business Reply Mail Processing Centre 969 Eastern           0.000000   
8                                 Caledonia-Fairbanks           0.000000   
9               Canada Post Gateway Processing Centre           0.000000   
10                                          Cedarbrae           0.000000   
11                                 Central Bay Street           0.000000   
1

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in to_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2              Café  0.04
3   Thai Restaurant  0.04
4  Sushi Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, South Steeles, Thistletown, Silverstone----
            venue  freq
0   Grocery Store   0.2
1  Discount Store   0.1
2     Pizza Place   0.1
3      Beer Store   0.1
4  Sandwich Place   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.10
1       Pet Store  0.05
2           Diner  0.05
3     Bridal Shop  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1                 Café  0.25
2 

#### Let's put that into a *pandas* dataframe

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neigh_venues_sorted = pd.DataFrame(columns=columns)
neigh_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neigh_venues_sorted.head()

Neighbourhood  \
0                           Adelaide, Richmond, King   
1                                          Agincourt   
2  Albion Gardens, Beaumond Heights, Humbergate, ...   
3    Bathurst Manor, Wilson Heights, Downsview North   
4                                    Bayview Village   

       1st Most Common Venue      2nd Most Common Venue 3rd Most Common Venue  \
0                Coffee Shop                 Restaurant       Thai Restaurant   
1  Latin American Restaurant               Skating Rink                Lounge   
2              Grocery Store        Fried Chicken Joint        Sandwich Place   
3                Coffee Shop  Middle Eastern Restaurant    Frozen Yogurt Shop   
4        Japanese Restaurant                       Café                  Bank   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0                  Café                   Bar            Steakhouse   
1        Breakfast Spot           Yoga Studio             Drugstore   
2        Discount Store  Fast Food Restaurant              Pharmacy   
3         Shopping Mall        Sandwich Place            Restaurant   
4    Chinese Restaurant   Distribution Center               Dog Run   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0      Sushi Restaurant          Concert Hall             Bookstore   
1   Distribution Center               Dog Run      Doner Restaurant   
2           Pizza Place   Japanese Restaurant            Beer Store   
3                 Diner         Deli / Bodega        Ice Cream Shop   
4      Doner Restaurant            Donut Shop             Drugstore   

  10th Most Common Venue  
0     Seafood Restaurant  
1             Donut Shop  
2                  Diner  
3            Supermarket  
4            Yoga Studio

## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

to_grouped_clustering =to_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print (len(kmeans.labels_))
kmeans.labels_[0:] 

100


array([3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3,
       3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 0, 2, 3, 3, 3, 3,
       3, 2, 3, 4, 3, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 2, 3], dtype=int32)

In [37]:
# add clustering labels
neigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = tzips_wrg2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(neigh_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged.head() # check the last columns!

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Port Union, Rouge Hill, Highland Creek  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353             4.0  Fast Food Restaurant    Dim Sum Restaurant   
1 -79.160497             3.0                   Bar         Moving Target   
2 -79.188711             3.0   Rental Car Location          Intersection   
3 -79.216917             3.0           Coffee Shop     Korean Restaurant   
4 -79.239476             3.0      Hakka Restaurant                Bakery   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0    Falafel Restaurant           Event Space  Ethiopian Restaurant   
1        History Museum           Yoga Studio            Donut Shop   
2        Breakfast Spot                   Spa    Mexican Restaurant   
3           Yoga Studio   Dumpling Restaurant   Distribution Center   
4  Caribbean Restaurant       Thai Restaurant    Athletics & Sports   

  6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
0   Empanada Restaurant     Electronics Store  Eastern European Restaurant   
1        Discount Store   Distribution Center                      Dog Run   
2     Electronics Store                  Bank               Medical Center   
3               Dog Run      Doner Restaurant                   Donut Shop   
4   Fried Chicken Joint           Gas Station                         Bank   

  9th Most Common Venue 10th Most Common Venue  
0   Dumpling Restaurant              Drugstore  
1      Doner Restaurant              Drugstore  
2   Distribution Center                Dog Run  
3             Drugstore      Electronics Store  
4             Drugstore             Donut Shop

### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [38]:
# Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
91   Etobicoke             0.0             Locksmith        Baseball Field   
97  North York             0.0        Baseball Field           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
91           Yoga Studio   Dumpling Restaurant   Distribution Center   
97             Drugstore        Discount Store   Distribution Center   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
91               Dog Run      Doner Restaurant            Donut Shop   
97               Dog Run      Doner Restaurant            Donut Shop   

   9th Most Common Venue       10th Most Common Venue  
91             Drugstore  Eastern European Restaurant  
97   Dumpling Restaurant         Fast Food Restaurant

In [39]:
# Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
21  North York             1.0                   Gym    Dim Sum Restaurant   
98        York             1.0     Convenience Store           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
21    Falafel Restaurant           Event Space  Ethiopian Restaurant   
98   Dumpling Restaurant        Discount Store   Distribution Center   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
21   Empanada Restaurant     Electronics Store  Eastern European Restaurant   
98               Dog Run      Doner Restaurant                   Donut Shop   

   9th Most Common Venue       10th Most Common Venue  
21   Dumpling Restaurant                    Drugstore  
98             Drugstore  Eastern European Restaurant

In [40]:
# Cluster 3
to_merged.loc[to_merged['Cluster Labels'] == 2, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
5        Scarborough             2.0  Construction & Landscaping   
14       Scarborough             2.0                        Park   
20        North York             2.0                   Cafeteria   
23        North York             2.0                        Park   
25        North York             2.0                        Park   
30        North York             2.0                        Park   
40         East York             2.0                        Park   
44   Central Toronto             2.0                        Park   
50  Downtown Toronto             2.0                        Park   
74              York             2.0                        Park   
79        North York             2.0                        Park   
90         Etobicoke             2.0                        Park   

   2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
5             Playground                 Yoga Studio             Drugstore   
14            Playground                 Yoga Studio            Donut Shop   
20                  Park                   Drugstore                 Diner   
23                  Bank           Convenience Store           Yoga Studio   
25     Food & Drink Shop                       Diner        Discount Store   
30               Airport  Construction & Landscaping           Yoga Studio   
40         Metro Station           Convenience Store            Donut Shop   
44           Swim School                    Bus Line           Yoga Studio   
50                 Trail                  Playground           Yoga Studio   
74         Women's Store                      Market           Yoga Studio   
79                Bakery  Construction & Landscaping           Yoga Studio   
90                 River                 Yoga Studio      Doner Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5         Discount Store   Distribution Center               Dog Run   
14    Dim Sum Restaurant                 Diner        Discount Store   
20        Discount Store   Distribution Center               Dog Run   
23             Drugstore        Discount Store   Distribution Center   
25   Distribution Center               Dog Run      Doner Restaurant   
30             Drugstore        Discount Store   Distribution Center   
40                 Diner        Discount Store   Distribution Center   
44            Donut Shop        Discount Store   Distribution Center   
50    Dim Sum Restaurant                 Diner        Discount Store   
74            Donut Shop                 Diner        Discount Store   
79             Drugstore        Discount Store   Distribution Center   
90    Dim Sum Restaurant                 Diner        Discount Store   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
5       Doner Restaurant            Donut Shop  Eastern European Restaurant  
14   Distribution Center               Dog Run             Doner Restaurant  
20      Doner Restaurant            Donut Shop                  Yoga Studio  
23               Dog Run      Doner Restaurant                   Donut Shop  
25            Donut Shop             Drugstore               Farmers Market  
30               Dog Run      Doner Restaurant                   Donut Shop  
40               Dog Run      Doner Restaurant                    Drugstore  
44               Dog Run      Doner Restaurant                    Drugstore  
50   Distribution Center               Dog Run             Doner Restaurant  
74   Distribution Center               Dog Run             Doner Restaurant  
79               Dog Run      Doner Restaurant                   Donut Shop  
90   Distribution Center               Dog Run                    Drugstore

In [41]:
# Cluster 4
to_merged.loc[to_merged['Cluster Labels'] == 3, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
1         Scarborough             3.0                        Bar   
2         Scarborough             3.0        Rental Car Location   
3         Scarborough             3.0                Coffee Shop   
4         Scarborough             3.0           Hakka Restaurant   
6         Scarborough             3.0             Discount Store   
7         Scarborough             3.0                     Bakery   
8         Scarborough             3.0        American Restaurant   
9         Scarborough             3.0            College Stadium   
10        Scarborough             3.0          Indian Restaurant   
11        Scarborough             3.0                 Smoke Shop   
12        Scarborough             3.0  Latin American Restaurant   
13        Scarborough             3.0                   Pharmacy   
15        Scarborough             3.0              Grocery Store   
17         North York             3.0   Mediterranean Restaurant   
18         North York             3.0             Clothing Store   
19         North York             3.0        Japanese Restaurant   
22         North York             3.0           Ramen Restaurant   
24         North York             3.0               Home Service   
26         North York             3.0        Japanese Restaurant   
27         North York             3.0                 Restaurant   
28         North York             3.0                Coffee Shop   
29         North York             3.0         Miscellaneous Shop   
31         North York             3.0              Grocery Store   
32         North York             3.0           Business Service   
33         North York             3.0               Liquor Store   
34         North York             3.0               Intersection   
35          East York             3.0                Pizza Place   
36          East York             3.0             Cosmetics Shop   
37       East Toronto             3.0                      Trail   
38          East York             3.0                Coffee Shop   
39          East York             3.0          Indian Restaurant   
41       East Toronto             3.0           Greek Restaurant   
42       East Toronto             3.0             Sandwich Place   
43       East Toronto             3.0                       Café   
45    Central Toronto             3.0           Department Store   
46    Central Toronto             3.0                Coffee Shop   
47    Central Toronto             3.0             Sandwich Place   
48    Central Toronto             3.0                Summer Camp   
49    Central Toronto             3.0                Coffee Shop   
51   Downtown Toronto             3.0                 Restaurant   
52   Downtown Toronto             3.0                Coffee Shop   
53   Downtown Toronto             3.0                Coffee Shop   
54   Downtown Toronto             3.0                Coffee Shop   
55   Downtown Toronto             3.0                Coffee Shop   
56   Downtown Toronto             3.0                Coffee Shop   
57   Downtown Toronto             3.0                Coffee Shop   
58   Downtown Toronto             3.0                Coffee Shop   
59   Downtown Toronto             3.0                Coffee Shop   
60   Downtown Toronto             3.0                Coffee Shop   
61   Downtown Toronto             3.0                Coffee Shop   
62         North York             3.0         Italian Restaurant   
63    Central Toronto             3.0                     Garden   
64    Central Toronto             3.0         Mexican Restaurant   
65    Central Toronto             3.0                       Café   
66   Downtown Toronto             3.0                       Café   
67   Downtown Toronto             3.0                        Bar   
68   Downtown Toronto             3.0            Airport Service   
69   Downtown Toronto             3.0                Coffee Shop   
70

In [42]:
# Cluster 5
to_merged.loc[to_merged['Cluster Labels'] == 4, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0  Scarborough             4.0  Fast Food Restaurant    Dim Sum Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0    Falafel Restaurant           Event Space  Ethiopian Restaurant   

  6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
0   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0   Dumpling Restaurant              Drugstore